In [2]:
from lazypredict.Supervised import LazyRegressor
import pandas as pd
from sklearn import datasets
from sklearn.utils import shuffle
import numpy as np
from sklearn.preprocessing import (LabelEncoder, StandardScaler, 
                                   PowerTransformer, QuantileTransformer, 
                                   OrdinalEncoder, OneHotEncoder, MinMaxScaler)
from sklearn.metrics import *
from sklearn.model_selection import (cross_val_score, cross_validate, 
                                     GridSearchCV, KFold, 
                                     RandomizedSearchCV, train_test_split,
                                     learning_curve, LearningCurveDisplay)
from sklearn.utils.estimator_checks import check_estimator
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt
import pickle
import shap
import seaborn as sns
import math
from scipy.stats import randint, uniform
# esto para destransformar features del boxcox
def inverse_bc(scaler, feature, position):
    if scaler.lambdas_[position] == 0:
        return np.exp(feature)
    else:
        return (feature * scaler.lambdas_[position] + 1) ** (1 / scaler.lambdas__[position])


In [11]:
data = pd.read_csv("data_model_complete.csv")

data_model_pwtr = data.copy()


toscale_features = ['Ncasos','sars_loess']

# for any reason i get negative 0s in some rows || plus i cannot use powertransform with 0s, i change to .1s
data_model_pwtr[toscale_features] = data_model_pwtr[toscale_features].applymap(lambda x: 1 if x <= 0 else x)

pow_scaler = PowerTransformer(method='box-cox')

data_model_pwtr[['Ncasos_pwtr', 'sars_pwtr']] = pow_scaler.fit_transform(data_model_pwtr[toscale_features])

In [13]:
data.columns

Index(['Week', 'Mun', 'Ncasos', 'Year', 'Ncasos_lead', 'Ncasos_lag',
       'Ncasos_lagroll_15', 'Ncasos_lagroll_30', 'EDAR', 'nHab', 'N1', 'N2',
       'IP4', 'E', 'caudal', 'lluvia', 'Fecha', 'sars2', 'sars', 'n1_fill',
       'f_num', 'n1_loess', 'n1_loess_15', 'n1_loess_30', 'sars_loess',
       'sars_loess_15', 'sars_loess_30', 'sars2_loess', 'sars2_loess_15',
       'sars2_loess_30', 'Month'],
      dtype='object')

In [16]:
data_pwtr = data_model_pwtr[toscale_features + ['Ncasos_pwtr', 'sars_pwtr', 'nHab']] # por si me sigue dando por usar los habitantes en la vaina
data_pwtr.to_csv('data_model_pwtr.csv')